# Tokenization
- The Byte Pair Encoding algorithm is a method for tokenization introduced in paper [Laungague Models are Unsupervised Miltitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). See section Input Representation which is where tokenization is covered.
- token is a fundamental unit of large language models; tokenization is the process of converting a string of text into tokens and vice cersa
- see [llama 2 paper](https://arxiv.org/pdf/2307.09288). They trained on 2 trillion tokens of data. 
- [TikTokenizer App](https://tiktokenizer.vercel.app/)

# Tokenization is at the heart of LLM issues
- LLMs can't spell words - this is because characters are chunked up into tokens. some represent many characters. for example '.DefaultCellStyle' represnted by a single token in cl100k_base model. When asking chatGPT "How many letter's l are in the word '.DefaultCellStyle'", it responds with 3.
- LLMs can't do simple string processing tasks like reverse a string
- LLMs are worse at non-English languages. - Dataset is larger in English, aka sees less non-English language during training. and tokenizer is not trained sufficiently on non-English data. There are longer tokens for English, aka uses less tokens.
- LLMs are bad at simple arithmetic. - That has to do with the tokenization of numbers. [Integer Tokenization is insane](https://www.beren.io/2023-02-04-Integer-tokenization-is-insane/). 
- GPT-2 had trouble coding in Python. - handling the whitespaces for gpt-2 was a tokenization bug
- LLM abruptly halts when sees text '<|endoftext|>'
- weird warning about training whitespace - occurs when base model is completing. partial token issue. tiktoken has a ton of code dealing with unstable tokens. 
- what if ask the LLM about '[SolidGoldMagikarp](https://www.lesswrong.com/posts/aPeJE8bSo6rAFoLqg/solidgoldmagikarp-plus-prompt-generation)'? - SolidGoldMagikarp is the name of a reddit user. The tokenization dataset had a lot of reddit data. This user posted a lot. So in tokenizer, that word got a dedicated token. Later whn the model was trained, the reddit data was not part of the training set, aka never seen during training, so it was never used in the emebdding table, aka completely untrained. At test time, when you invoke the SolidGoldMagikarp token, which is completely untrained, feed it into a transformer, you'll get completely undefined behavior. neato...
- prefer to use YAML over JSON with LLM. why? JSON is very dense in tokens. and yaml is efficient. you should always care about tokenization density. 
- LLM is not actually end-to-end language modeling

# in practice
- re-use GPT-4 tokens and tiktoken. already efficient for inference
- if you're training a vocab, ok to use BPE with sentencepiece. be careful with the settings.
- another lib: minbpe

In [2]:
# naiive character level tokenizer:

# read in tiny shakespeare as stirng
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in chracters:", len(text), "\n-----\n")
print("first 100 characters: ", text[:100])

# get unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)  # this is the numebr of possible options for next characer in sequence

# create mapping from chracters to integers and back
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: takes a list of integers, output a string

# encode the entire txt dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print("first 100 tokens: ", data[:100])

length of dataset in chracters: 1115394 
-----

first 100 characters:  First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
first 100 tokens:  tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


# Unicode
- we want to take strings and feed them into language models. So we need to tokenize strings into some integers in a fixed vocabulary. We use integers to lookup into a lookup table of vectors and feed those vectors into the Transformer as an input.
- we want to support different languages and special characters (like emoji).
- in Python, strings are immutable sequences of Unicode code points. [Unicode](https://en.wikipedia.org/wiki/Unicode#Versions) standard is a definition of ~150k characters, what they look like and what integers represent those characters. 
- how do we access the unicode code point for a character in Python? Use the `ord()` function.
- Unicode standard is alive and keeps changing. Aka it is not stable and should not be used directly.

# Encodings
- Unicode standard defines 3 encodings: utf-8, utf-16, utf-32.  Encodings is the way to take unicode text and translate it into binary data or byte streams. [utf-8](https://en.wikipedia.org/wiki/UTF-8#Description) is the most common. 
- utf-8 takes the code points and translates it to a byte stream. Each code point is translated to a byte stream that is 1 to 4 bytes long. Aka it is a variable length encoding.
- see pros and cons in [blog post](https://www.reedbeta.com/blog/programmers-intro-to-unicode/) and [utf-8 manifesto](https://utf8everywhere.org/).
- utf-8 is the only encoding that is backward compatible to the much simpler ASCII encoding of text.
- utf-16 and utf-32 can be wasteful encodings. simple ascii characters encode into something like zero number zero other number.

# Vocabulary Size
- utf-8 byte stream implies a vocabulary length of 256 possible tokens. This is too small. All the text would be stretched out over very long sequences. Emebdding table and prediction will be tiny. But we have finite context length  and attention that can be supported in a transformer for computational reasons. It is inefficient for sufficiently long text for the purpose of predicting the next token. 
- We want much larger vocabulary size whose size is tuned as a hyperparameter and we want to stick with the utf-8 encoding. 
- Turn to the byte-pair encoding algorithm
- see paper [Megabyte: Predicting million-byte sequences with mutli-scale transformers](https://arxiv.org/pdf/2305.07185). It modifies transfomer architecture into a hierarchical one in order to be able to feed in raw bytes. It stated that it established the viability of a tokenization-free autorgressive sequence modeling at scale.

In [3]:
string = "こんにちは 👋 (hello in Japanese!)"
print(f'{string=}')
code_points = [(c, ord(c)) for c in string]
print(f'{code_points=}')
encoding = string.encode('utf-8')  # returns bytes objects
print(f'{encoding=}')
raw_bytes = list(encoding)
print(f'{raw_bytes=}')

string='こんにちは 👋 (hello in Japanese!)'
code_points=[('こ', 12371), ('ん', 12435), ('に', 12395), ('ち', 12385), ('は', 12399), (' ', 32), ('👋', 128075), (' ', 32), ('(', 40), ('h', 104), ('e', 101), ('l', 108), ('l', 108), ('o', 111), (' ', 32), ('i', 105), ('n', 110), (' ', 32), ('J', 74), ('a', 97), ('p', 112), ('a', 97), ('n', 110), ('e', 101), ('s', 115), ('e', 101), ('!', 33), (')', 41)]
encoding=b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf \xf0\x9f\x91\x8b (hello in Japanese!)'
raw_bytes=[227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 32, 240, 159, 145, 139, 32, 40, 104, 101, 108, 108, 111, 32, 105, 110, 32, 74, 97, 112, 97, 110, 101, 115, 101, 33, 41]


# [Byte-Pair Encoding Algorithm](https://en.wikipedia.org/wiki/Byte_pair_encoding)
- allows us to compress the byte sequences to a variable amount
- "The original BPE algorithm operates by iteratively replacing the most common contiguous sequences of characters in a target text with unused 'placeholder' bytes. The iteration ends when no sequences can be found, leaving the target text effectively compressed."
- "Compared to the original BPE, the modified BPE does not aim to maximally compress text, but rather, to encode plaintext into "tokens", which are natural numbers."
- how many iterations do we do? that's a hyperparameter. The more iterations, the bigger the vocabulary size will be and the shorter the sequence will be. There is a sweet spot for this in practice.
- GPT-4 uses roughly ~100k tokens for its LLM.


In [4]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
code_points = text.encode('utf-8')  # raw bytes
tokens = list(map(int, code_points))  # convert to list of integers in range 0 to 255
print(f'---\n{text}\n---\nlength of text: {len(text)} code points\n---\n{tokens}\n---\nlength of tokens: {len(tokens)}\n\n')
# simple ASCII characters become 1 byte and complex unicode characters can become multiple bytes (up to 4)

from collections import defaultdict
def get_stats(ids):
    counts = defaultdict(int)
    for pair in zip(ids, ids[1:]):  # Pythonic way to iterate over consecutive elements
        counts[pair] += 1
    return counts

stats = get_stats(tokens)
print("---\nstats sorted: ", sorted([(v,k) for k,v in stats.items()], reverse=True))  # by default, Python sorts on the first element
print(f'---\n{chr(101)=}, {chr(32)=}\n\n')  # opposite of ord is chr in Python; a lot of words end in 'e'!

# get highest ranking pair
top_pair = max(stats, key=stats.get)  # provides a function to rank keys, stats.get returns the value, aka stats.get((101,32)) = 20

def merge(ids, pair, idx):
    # in the list of ints (ids), replace all consecutive occurrences of pair with the new token idx
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:  # they match
            newids.append(idx)
            i += 2
        else:  # only one id left OR two ids left and they don't match
            newids.append(ids[i])
            i += 1
    return newids

print(f'---\nexample of merge: {merge([5, 6, 6, 7, 9, 1], (6, 7), 99)=}')
tokens_2 = merge(tokens, top_pair, 256)
print(f'---\nmerge(tokens, top_pair, 256): {tokens_2}')
print(f'---\nlength of new tokens: {len(tokens_2)}')

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
---
length of text: 533 code points
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159,

In [25]:
def byte_pair_encoding(ids, num_merges):
    merges = {}  # pair tuple : id
    for i in range(num_merges):
        stats = get_stats(ids)  # returns dict of pair tuple : count
        top_pair = max(stats, key=stats.get)  # returns pair tuple with maximum count
        idx = 256 + i
        print(f'merging pair {top_pair} into a new token {idx}')
        ids = merge(ids, top_pair, idx)
        merges[top_pair] = idx
    return ids, merges

# -----------------------------------------------------------------

# use longer training text for more representative token statistics
with open('blog.txt', 'r', encoding='utf-8') as f:
    blog = f.read()
code_points = blog.encode('utf-8')  # raw bytes
tokens = list(map(int, code_points))  # convert to list of integers in range 0 to 255
print(f'---\n{blog[:100]=}\n---\nlength of text: {len(blog)} code points\n---\n{tokens}\n---\nlength of tokens: {len(tokens)}\n\n')

vocab_size = 276  # the desired final vocab size, set based on performance
num_merges = vocab_size - 256  # we already have 256 tokens from raw bytes
ids = list(tokens)

newtokens, merges = byte_pair_encoding(ids, num_merges)
print(f'---\nafter BPE: {newtokens}\n---\nlength of tokens: {len(newtokens)}')
print("---\nmerges: ", merges, "\n\n")

print(f'---compression ratio: {len(tokens)} / {len(newtokens)} = {len(tokens)/len(newtokens):.2f}X')


---
blog[:100]='Nathan Reed\nBlog Stuff I’ve Made Talks About Me\nThe Many Meanings of “Shader”Quadrilateral Interpola'
---
length of text: 23927 code points
---
[78, 97, 116, 104, 97, 110, 32, 82, 101, 101, 100, 10, 66, 108, 111, 103, 32, 83, 116, 117, 102, 102, 32, 73, 226, 128, 153, 118, 101, 32, 77, 97, 100, 101, 32, 84, 97, 108, 107, 115, 32, 65, 98, 111, 117, 116, 32, 77, 101, 10, 84, 104, 101, 32, 77, 97, 110, 121, 32, 77, 101, 97, 110, 105, 110, 103, 115, 32, 111, 102, 32, 226, 128, 156, 83, 104, 97, 100, 101, 114, 226, 128, 157, 81, 117, 97, 100, 114, 105, 108, 97, 116, 101, 114, 97, 108, 32, 73, 110, 116, 101, 114, 112, 111, 108, 97, 116, 105, 111, 110, 44, 32, 80, 97, 114, 116, 32, 50, 10, 65, 32, 80, 114, 111, 103, 114, 97, 109, 109, 101, 114, 226, 128, 153, 115, 32, 73, 110, 116, 114, 111, 100, 117, 99, 116, 105, 111, 110, 32, 116, 111, 32, 85, 110, 105, 99, 111, 100, 101, 10, 77, 97, 114, 99, 104, 32, 51, 44, 32, 50, 48, 49, 55, 32, 194, 183, 32, 67, 111, 100, 105, 110, 1

# Notes
- The Tokenizer is a completely separate module from the LLM. The Tokenizer has its own training dataset of text on which you train the vocabulary using the BPE algorithm. It then translates back and forth between raw text and sequnces of tokens. The LLM only deals with tokens, never with any text.
- You may want different training set for the tokenizer and the LLM. The tokenizer should be exposed to different lanugaes and special characters and different amounts of code. The amount of different languages determines the numnber of merges and therfore determines the density for the type of data in the token space. 
- The Tokenizer takes raw text and translates it into a token sequence and vice versa. So now how to do the encoding and decoding steps? Use the merges.

# Encoding and Decoding
- Given a sequence of integers in the range [0, vocab_size], what is the text?


In [71]:
def decode_2(ids, merges):
    # given ids (list of integers), return string
    m = merges.copy()
    
    # merges is dict of tuple pair to token idx
    # if token idx in ids, replace all instances of idx with tuple pair
    # and remove pair from merges dict
    # if token idx not in ids, move on to next idx in merges, do not remove from merges
    # repeat until merges dict is empty
    print("merges:", m)
    print("ids:", ids)
    found = True
    
    while m and found:
        print(f'{len(m)=}, {len(ids)=}')
        found = False
        for pair, idx in m.items():
            if idx in ids:
                print("found ", idx, ", will replace with pair", pair)
                found = True
                newids = []
                for id in ids:
                    if id == idx:
                        newids.append(pair[0])
                        newids.append(pair[1])
                    else:
                        newids.append(id)
                ids = newids
                del m[pair]
                break
    
    return ''.join([chr(id) for id in ids])

print("\n", decode_2([128], merges))
print('--------------------------')
print("\n", decode_2([97, 257, 256], merges))



merges: {(101, 32): 256, (105, 110): 257, (115, 32): 258, (116, 104): 259, (101, 114): 260, (99, 111): 261, (116, 32): 262, (226, 128): 263, (44, 32): 264, (97, 110): 265, (111, 114): 266, (100, 32): 267, (97, 114): 268, (101, 110): 269, (257, 103): 270, (261, 100): 271, (121, 32): 272, (97, 108): 273, (111, 110): 274, (259, 256): 275}
ids: [128]
len(m)=20, len(ids)=1

 
--------------------------
merges: {(101, 32): 256, (105, 110): 257, (115, 32): 258, (116, 104): 259, (101, 114): 260, (99, 111): 261, (116, 32): 262, (226, 128): 263, (44, 32): 264, (97, 110): 265, (111, 114): 266, (100, 32): 267, (97, 114): 268, (101, 110): 269, (257, 103): 270, (261, 100): 271, (121, 32): 272, (97, 108): 273, (111, 110): 274, (259, 256): 275}
ids: [97, 257, 256]
len(m)=20, len(ids)=3
found  256 , will replace with pair (101, 32)
len(m)=19, len(ids)=4
found  257 , will replace with pair (105, 110)
len(m)=18, len(ids)=5

 aine 


In [72]:
# DECODING - token sequence to raw text

# create pre-processing variable called vocab, a mapping from token id to the bytes object for that token
vocab = {idx: bytes([idx]) for idx in range(256)}
# go in order in which items were inserted in the dicitonary, guaranteed with Python 3.7,  populate the vocab list
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]  # concat 2 bytes objects

def decode(ids):
    # given ids (list of integers), return string
    # iterate over the ids, use vocab to get their bytes, concat all the bytes, tokens are nor raw bytes
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode('utf-8', errors='replace')  # errors replace is standard practice for LLMs, also done by OpenAI
    return text

print(decode([128]))  # without setting errors to replace in decode, get: UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
# not all byte sequences are valid utf-8
print('--------------------------')
print(decode([97, 257, 256]))


�
--------------------------
aine 


In [73]:
merges
# merges was built top to bottom
# note the order, this is the order that we merged tokens
# so when encoding, we should also merge in the same order because later merges may rely on earlier merges

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (97, 108): 273,
 (111, 110): 274,
 (259, 256): 275}

In [74]:
# ENCODING - string to tokens
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = text.encode('utf-8')  # gets raw bytes 
    tokens = list(map(int, tokens))
    while len(tokens) >= 2:  # in case text is 1 character long
        stats = get_stats(tokens)  # want raw pairs in this sequence, aka the keys in this dictionary
        # find key inside stats that has the lowest index in the merges dictionary
        # call min on iterator stats, looks at keys of stats dictionary, which are pairs
        # merges.get(pair)
        # so for any pair in stats, we will look at the pair in merges, and get its new token id
        # default inf if pair is not in merges
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))  # gets next eligible pair
        if pair not in merges:
            break  # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(f'{encode("hello world")=}')
print(f'{decode([104, 101, 108, 108, 111, 32, 119, 266, 108, 100])=}')
print(f'{encode("h")=}')

encode("hello world")=[104, 101, 108, 108, 111, 32, 119, 266, 108, 100]
decode([104, 101, 108, 108, 111, 32, 119, 266, 108, 100])='hello world'
encode("h")=[104]


In [75]:
# Testing
# some byte streams are not valid utf-8 streans, so we can test this only 1 way
print(f'{decode(encode('hello world')) == 'hello world'}')
text2 = decode(encode(text))
print(f'{text2 == text}')


True
True


# Force Splits using Regex Patterns (GPT series)
- look at the "Input Representation" section of the GPT-2 paper "Language Models are Unsupervised Multitask Learners". This discussed the tokenizer used for GPT-2. BPE uses a greedy based heuristic for building the token vocabulary. this is suboptimal for common words like "dog", "dog.", "dog,", "dog!" They prevent BPE from merging across character categories for any byte sequence. they add an exception for spaces.
- In [gpt-2's inference code for tokenizer](https://github.com/openai/gpt-2/blob/master/src/encoder.py), there is a [regex pattern](https://github.com/openai/gpt-2/blob/master/src/encoder.py#L53) that allows them to enforce rules for parts of text that will never be merged. 
```python
self.pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
```
- python has a built-in package called [**re**](https://docs.python.org/3/library/re.html) to work with regular expressions, 
- there is also the [**regex**](https://pypi.org/project/regex/) package, which is a little more powerful than **re**
- reference regular expressions [unicode categories](https://www.regular-expressions.info/unicode.html) 
- **?**: optional
- **+**: one or more
- **\p{L}** or **\p{Letter}**: any kind of letter from any language.
- **\p{N}** or **\p{Number}**: any kind of numeric character in any script.
- **[^\s\p{L}\p{N}]+**: one or more of something that is not a letter, number, or a space; this is trying to match punctuation
- **\s+(?!\S)**: matching whitespace up to but not inlcuding the last whitespace character; this is a negative look ahead insertion
- **\s+**: whitespace characters; catches any trailing spaces
- openai gpt2 has some additional rules, they don't just do chunking with regex and then BPE
- the vocabulary size for gp2 is roughly 50k

In [76]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello world123 how are you"))  # results in ['Hello', ' world', '123', ' how', ' are', ' you']
# BPE is applied to this list, you will only find merges on this list
# so effectively, you will never merge the e in are with the space before you
# using the regex pattern to chunk up the text is one way to enforce that some merges are not allowed
print(re.findall(gpt2pat, "Hello've world123 how are      you!!!?   "))

fizzbuzz_example = """
for num in range(1, 101):
    if num % 3 == 0 and num % 5 == 0:
        print('FizzBuzz')
    elif num % 3 == 0:
        print('Fizz')
    elif num % 5 == 0:
        print('Buzz')
    else:
        print(num)
"""
print(re.findall(gpt2pat, fizzbuzz_example))

['Hello', ' world', '123', ' how', ' are', ' you']
['Hello', "'ve", ' world', '123', ' how', ' are', '     ', ' you', '!!!?', '   ']
['\n', 'for', ' num', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' num', ' %', ' 3', ' ==', ' 0', ' and', ' num', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', "('", 'FizzBuzz', "')", '\n   ', ' elif', ' num', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', "('", 'Fizz', "')", '\n   ', ' elif', ' num', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', "('", 'Buzz', "')", '\n   ', ' else', ':', '\n       ', ' print', '(', 'num', ')', '\n']


# [tiktoken](https://github.com/openai/tiktoken)
- tiktoken is OpenAI's official library for tokenization
- gpt-4's tokenizer (c1100k_base) has a different [regular expression](https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py#L89) for chunking up text
```re
r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}++|\p{N}{1,3}+| ?[^\s\p{L}\p{N}]++[\r\n]*+|\s++$|\s*[\r\n]|\s+(?!\S)|\s"""
```
- **i**: case insensitive match
- **'(?i:[sdmt]|ll|ve|re)**: apostrophe followed by ll, ve, etc, case insensitive
- **\p{N}{1,3}+**: match 1-3 numbers. so they will never merge numbers that are more than 3 digits, only up to 3 digits! this is to prevent tokens that are very long number sequences
- vocabulary size for gpt-4 is roughly 100k

In [77]:
import tiktoken
# this is inference code for tokenization, not for training ***

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("       hello world!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("       hello world!!!"))


[220, 220, 220, 220, 220, 220, 23748, 995, 10185]
[996, 24748, 1917, 12340]


# [gpt-2's encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- **get_encoder** function loads 2 files, encoder.json (the vocabulary) and vocab.bpe (the merges), does some light processing, and then they call the Encoder object, which is the tokenizer
- gpt-2 has a encoder, decoder, and byte_encoder and byte_decoder. you do byte_encode, then encode, then decode, then byte_decode, stacked serially on top of each other. 
- the Encoder's **bpe** function is the byte-pair algorithm. In the while loop, first identify the bigram (aka top pair to merge next). inner loop merges pairs whenever found. and the outer loop repeats until run out of possible merges.
- the Encoder also has an **encode** and **decode** function. 

# Special Tokens
- in addition to tokens that come from raw bytes and bpe merges, we can insert tokens used to delimit parts of data or introduce a special structure of the token streams
- there are 256 raw byte tokens. the mapping table (encoder) is of length 50257. so openai did 50,000 merges. and there is also 1 speical token
- the special token is "<|endoftext|>", and it is the very last token with an id of 50256
- the endoftext special token is used to delmit documents in the training set
- this special token doesn't go through the bpe merges, there is special case instructions for handling special token
- these instructions are absent from gpt-2's encoder.py, but [tiktoken's encode](https://github.com/openai/tiktoken/blob/main/src/lib.rs) function in Rust does
- when you add special tokens, you have to do model surgey to the Transformer and all the parameters involved, because you are adding an integer, so embedding matrix has to be extended by adding a row, init with small random numbers, because we now need a vector that stands for that special token. you also have to go to the final layer of the transformer, and make sure projection at the very end into the classifier is extended by 1 as well. So in summary, there is some model surgery that must occur coupled with the tokenization changes in order to add special tokens. This is needed to fine-tune the model, aka going from a base model to a chat model like chatGpt. 

## [gpt-4's special tokens](https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py#L81)
- see base cl100k_base, special tokens includes ENDOFTEXT, FIM_PREFIX, FIM_MIDDLE, FIM_SUFFIX, ENDOFPROMPT where
```python
ENDOFTEXT = "<|endoftext|>"
FIM_PREFIX = "<|fim_prefix|>"
FIM_MIDDLE = "<|fim_middle|>"
FIM_SUFFIX = "<|fim_suffix|>"
ENDOFPROMPT = "<|endofprompt|>"
```
- FIM is short for fill in the middle
- FIM reference paper: [Efficient Training of Language Models to Fill in the Middle](https://arxiv.org/pdf/2207.14255)

## fine-tuning
- fine-tuned models will have more special tokens than the base models
- they are used to keep track of messages and the flow of the messsages
- see gpt-3.5-turbo model's tokens sample on [tiktokenizer](https://tiktokenizer.vercel.app/?model=gpt-3.5-turbo). this includes special tokens **<im_start>**, **<im_end>**
- **im** stands for imaginary monologe

# [Extending tiktoken](https://github.com/openai/tiktoken/tree/main?tab=readme-ov-file#extending-tiktoken)
- you can fork a base model like cl100k_base and add special tokens to it


In [ ]:
import json

with open('encoder.json', 'r') as f:
    encoder = json.load(f)  # like the vocabulary, character : integer though

with open('vocab.bpe', 'r', encoding='utf-8') as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]

# using these 2 variables representing the tokenizer, you can do encoding and 

print(f'{encoder["!"]=}')
print(f'{encoder["m"]=}')
print(f'{encoder["<|endoftext|>"]=}')
print(f'{len(encoder.keys())=}')

print(f'\n{bpe_merges[:100]=}')
print(f'{len(bpe_merges)=}')

encoder["!"]=0
encoder["m"]=76
encoder["<|endoftext|>"]=50256
len(encoder.keys())=50257

bpe_merges[:100]=[('Ġ', 't'), ('Ġ', 'a'), ('h', 'e'), ('i', 'n'), ('r', 'e'), ('o', 'n'), ('Ġt', 'he'), ('e', 'r'), ('Ġ', 's'), ('a', 't'), ('Ġ', 'w'), ('Ġ', 'o'), ('e', 'n'), ('Ġ', 'c'), ('i', 't'), ('i', 's'), ('a', 'n'), ('o', 'r'), ('e', 's'), ('Ġ', 'b'), ('e', 'd'), ('Ġ', 'f'), ('in', 'g'), ('Ġ', 'p'), ('o', 'u'), ('Ġa', 'n'), ('a', 'l'), ('a', 'r'), ('Ġt', 'o'), ('Ġ', 'm'), ('Ġo', 'f'), ('Ġ', 'in'), ('Ġ', 'd'), ('Ġ', 'h'), ('Ġan', 'd'), ('i', 'c'), ('a', 's'), ('l', 'e'), ('Ġt', 'h'), ('i', 'on'), ('o', 'm'), ('l', 'l'), ('en', 't'), ('Ġ', 'n'), ('Ġ', 'l'), ('s', 't'), ('Ġ', 're'), ('v', 'e'), ('Ġ', 'e'), ('r', 'o'), ('l', 'y'), ('Ġb', 'e'), ('Ġ', 'g'), ('Ġ', 'T'), ('c', 't'), ('Ġ', 'S'), ('i', 'd'), ('o', 't'), ('Ġ', 'I'), ('u', 't'), ('e', 't'), ('Ġ', 'A'), ('Ġ', 'is'), ('Ġ', 'on'), ('i', 'm'), ('a', 'm'), ('o', 'w'), ('a', 'y'), ('a', 'd'), ('s', 'e'), ('Ġth', 'at'), ('Ġ', 'C'), ('i', 'g')

# [sentencepiece](https://github.com/google/sentencepiece)
- the tokenization used by OpenAI LLM
- unlike tiktoken, it can do both training and inference with BPE
- tikotken first encodes to utf-8 then BPE's bytes, sentencepice BPEs the code points, and optionally falls back to utf-8 bytes for rare code points, which then get translated to byte tokens. rarity is determined by a character_coverage hyperparameter
- sentencepiece [training options](https://github.com/google/sentencepiece/blob/master/doc/options.md) document and [raw proto](https://github.com/google/sentencepiece/blob/master/src/sentencepiece_model.proto) to represent the trainer spec
- 

In [88]:
import os
import sentencepiece as spm
tswizzy_love_story = "We were both young when I first saw you. I close my eyes and the flashback starts. I'm standin' there. On a balcony in summer air. See the lights, see the party, the ball gowns. See you make your way through the crowd. And say, 'Hello.' Little did I know that you were Romeo, you were throwin' pebbles. And my daddy said, 'Stay away from Juliet.' And I was cryin' on the staircase. Beggin' you, 'Please don't go, ' and I said. Romeo, take me somewhere we can be alone. I'll be waiting, all there's left to do is run. You'll be the prince and I'll be the princess. It's a love story, baby, just say, 'Yes'"
with open('twsizzy.txt', 'w', encoding='utf-8') as f:
    f.write(tswizzy_love_story)

# llama 2 training like options
options = dict(
    # input spec
    input = "twsizzy.txt",
    input_format = "text",
    # output spec
    model_prefix = "tok400",  # output filename prefix
    # algorithm spec
    # bpe algorithm
    model_type = "bpe",
    vocab_size = 400,
    # normalization
    normalization_rule_name = "identity",
    remove_extra_whitespaces = False,
    input_sentence_size = 200000000,  # max number of training sentences
    max_sentence_length = 4192,  # max number of bytes per sentence
    seed_sentencepiece_size = 1000000,
    shuffle_input_sentence = True,
    # rare word treatment aka rare code points
    character_coverage = 0.99995,
    byte_fallback = True,
    # merge rules
    split_digits = True,
    split_by_unicode_script = True,
    split_by_whitespace = True,
    split_by_number = True,
    max_sentencepiece_length = 16,
    add_dummy_prefix = True,
    allow_whitespace_only_pieces = True,
    # special tokens
    unk_id = 0,
    bos_id = 1,
    eos_id = 2,
    pad_id = -1,
    # systems
    num_threads = os.cpu_count()  # use all system resources
)

spm.SentencePieceTrainer.train(**options)  # this will create 2 files: tok400.model and tok400.vocab

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: twsizzy.txt
  input_format: text
  model_prefix: tok400
  model_type: BPE
  vocab_size: 400
  self_test_sample_size: 0
  character_coverage: 0.99995
  input_sentence_size: 200000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  d

425) LOG(INFO) Adding meta_piece: <0xA7>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xA8>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xA9>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAA>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAB>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAC>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAD>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAE>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xAF>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB0>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB1>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB2>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB3>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB4>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB5>
trainer_interface.cc(425) LOG(INFO) Adding meta_piece: <0xB6>
trainer_interface.cc(425) LOG

In [ ]:
sp = spm.SentencePieceProcessor()
# load model and inpsect vocabulary
sp.load("tok400.model")
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab
# notice order is special token, byte tokens, merged tokens, and then individual code point tokens
# the raw code point tokens are the ones encountered in the training set

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [ ]:
japanese_hello = "こんにちは 👋 (hello in Japanese!)"
ids = sp.encode(japanese_hello)
print(ids)
print([sp.id_to_piece(idx) for idx in ids])

#--------------------------
# option byte_fallback
# japanese characters were not part of the training set (which was part of a tswizzy song)
# so sentencepiece is encontering code points that it did not see during training time, aka those code points do not have an associated token, aka unknown tokens
# however, byte_fallback option was set to true in the options
# so sentencepiece takes the japanese characters, encodes it with utf-8, and then uses tokens to represent those bytes
# if byte_fallback in options was set to False, then all the unknown characters (not see in training set), would be labeled as the piece <unk>, token 0
# what does LLM do when unks are fed into it? llama correctly uses byte_fallback true
#--------------------------
# option add_dummy_prefix
# from proto definition: Adds dummy whitespace at the beginning of text in order to treat "world" in "world" and "hello world" in the same way.
# so as part of pre-processing, sentencepiece takes the string and add a space in front
# llama 2 uses this option
#--------------------------
# sentencepiece
# some footguns, commonly used in industry though
# some quirks with byte_fallback

[361, 230, 132, 150, 230, 133, 150, 230, 132, 174, 230, 132, 164, 230, 132, 178, 361, 243, 162, 148, 142, 361, 43, 372, 362, 272, 365, 361, 266, 361, 393, 363, 386, 334, 279, 362, 36, 44]
['▁', '<0xE3>', '<0x81>', '<0x93>', '<0xE3>', '<0x82>', '<0x93>', '<0xE3>', '<0x81>', '<0xAB>', '<0xE3>', '<0x81>', '<0xA1>', '<0xE3>', '<0x81>', '<0xAF>', '▁', '<0xF0>', '<0x9F>', '<0x91>', '<0x8B>', '▁', '<0x28>', 'h', 'e', 'll', 'o', '▁', 'in', '▁', 'J', 'a', 'p', 'an', 'es', 'e', '<0x21>', '<0x29>']


# Vocab size
- What should be the vocab size? How can I increase the vocab size?
- recall the transformer model designed in this repo, see transformer.py in gpt folder, vocab_size was set length of chars unique from tiny shakespeare, ~65
- where is vocab_size important in the transformer model? In the Tranformer model class init function, vocab_size is used in:
    1. the number of embeddings in the embedding lookup table in the transformer model, aka number of rows, each row/vector is of size n_embd (number of channels).
    - as vocab size grows, so does the embedding lookup table number of rows
    2. the number of out features in the last linear layer of the transfomer model.
    - this layer is used at the very end to produce the logits which become the probabilities for the next token in the sequence
- why can't vocab size be infinite? 
    1. token embedding table and linear layer would grow, aka more computation
    2. with a larger vocab size, every token will come up more rarely in the training data because there are a lot more tokens, so we'll see fewer examples for each individual token, so vectors assocaited with tokens will be undertrained since they do not come up often
    3. as vocab size grows, your sequence will shrink a lot, so we attend to more text, but you worry that too large a chunk will become sequeezed into a token. model will not have much time to think. basically squishing too much information into a token and then forward pass of transformer is not processing that information appropriately
- vocab_size is an empirical hyperparameter
- in state of the art architectures, vocab_size is usually 10,000s or 100,000
- to extend a pre-trained model's vocab_size? a lot of new special tokens get introduced on top of the base model to maintain the metadata/structure of conversation objects between user and assistant. maybe more special token for using browser, special tool or some special functionality. minor surgery:
    1. resizing: the embedding table, aka add rows, we would init parameters from scratch, aka small random numbers
    2. resizing: extend the weight inside the last linear layer to be able to calculate probabilities for new tokens
- see paper [Learning to Compress Prompts with Gist Tokens](https://arxiv.org/pdf/2304.08467) to see more of the design space for parameter efficiency techniques. this is a compression technqiue for compressing a long prompt into less tokens. use the shorter tokens in the context to stand in for that long prompt for almost identical performance.

# Multi-modal
- [Taming Transformers for High-Resolution Iamge Synthesis](https://arxiv.org/pdf/2012.09841)
- don't change the architecture, stick with the transformer
- [OpenAI Sora](https://openai.com/sora/) has visual patches, a way to chunk-ate videos

